In [1]:
# RUN ONCE PER SESSION
import os
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)

/Users/niche/Library/CloudStorage/OneDrive-VirginiaTech/_03_Papers/2023/cowsformer


In [2]:
import ultralytics
from ultralytics import YOLO

In [3]:
DIR_DATA = os.path.join(ROOT, "data")
DIR_MODEL = os.path.join(ROOT, "models")
PATH_MODEL = os.path.join(DIR_MODEL, "yolov8m.pt")
PATH_YAML = os.path.join(DIR_DATA, "cow100", "yolov5", "data.yaml")

## Shuffle YOLO dataset

In [4]:
from data.splitter.yolo import YOLO_Splitter

path_yolo = os.path.join(DIR_DATA, "cow100", "yolov5")
splitter = YOLO_Splitter(path_yolo, classes=["cow"])
splitter.shuffle_train_test()
splitter.write_dataset()

## Train YOLOv8 on YOLO dataset

In [13]:
from ultralytics import YOLO
from models.device import get_device

model = YOLO(PATH_MODEL)

In [18]:
model.train(data=PATH_YAML, batch=16, imgsz=640, device="mps")

New https://pypi.org/project/ultralytics/8.0.114 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.99 🚀 Python-3.9.16 torch-2.0.1 MPS
yolo/engine/trainer: task=detect, mode=train, model=/Users/niche/Library/CloudStorage/OneDrive-VirginiaTech/_03_Papers/2023/cowsformer/models/yolov8m.pt, data=/Users/niche/Library/CloudStorage/OneDrive-VirginiaTech/_03_Papers/2023/cowsformer/data/cow100/yolov5/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

In [4]:
path_best = os.path.join(ROOT, "runs", "detect", "train3", "weights", "best.pt")
model = YOLO(path_best)

## Evaluation

In [18]:
os.getcwd()

'/Users/niche/Library/CloudStorage/OneDrive-VirginiaTech/_03_Papers/2023/cowsformer'

In [23]:
metrics = model.val(split="test",
                    device="mps",
                    project="out",
                    name="val",
                    exist_ok=True)

Ultralytics YOLOv8.0.99 🚀 Python-3.9.16 torch-2.0.1 MPS
val: Scanning /Users/niche/Library/CloudStorage/OneDrive-VirginiaTech/_03_Papers/2023/cowsformer/data/cow100/yolov5/test/labels.cache... 22 images, 4 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:21<00:00, 10.81s/it]
                   all         22         81      0.853      0.859      0.895      0.574
Speed: 44.3ms preprocess, 33.9ms inference, 0.0ms loss, 2.9ms postprocess per image
Saving out/val/predictions.json...
Results saved to out/val


In [25]:
metrics

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x2f9e55820>
fitness: 0.6060137965615419
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.57396])
names: {0: 'cow'}
plot: True
results_dict: {'metrics/precision(B)': 0.8528940926773689, 'metrics/recall(B)': 0.8589738308262183, 'metrics/mAP50(B)': 0.8945327686378914, 'metrics/mAP50-95(B)': 0.573956132997503, 'fitness': 0.6060137965615419}
save_dir: PosixPath('out/val')
speed: {'preprocess': 44.253912839022554, 'inference': 33.89755162325772, 'loss': 0.0007802789861505682, 'postprocess': 2.889210527593439}

In [38]:
metrics.box.__dict__

{'p': array([    0.85289]),
 'r': array([    0.85897]),
 'f1': array([    0.85592]),
 'all_ap': array([[    0.89453,     0.86561,     0.81886,     0.79038,     0.75873,      0.6364,     0.55171,     0.30886,      0.1113,   0.0031808]]),
 'ap_class_index': array([0]),
 'nc': 1}

In [8]:
confusion_matrix = metrics.confusion_matrix.matrix #   conf=0.25, iou_thres=0.45
confusion_matrix

array([[         70,          13],
       [         11,           0]])

In [15]:
metrics.confusion_matrix.tp_fp()

(array([         70]), array([         13]))

In [16]:
n_cows = confusion_matrix[:, 0].sum().astype(int)
n_cows_missed = confusion_matrix[1, 0].sum().astype(int)
n_cows_false = confusion_matrix[0, 1:].sum().astype(int)
print("n cows: ", n_cows)
print("n cows missed: ", n_cows_missed)
print("n cows false positive: ", n_cows_false)


n cows:  81
n cows missed:  11
n cows false positive:  13


In [10]:
(n_cows - n_cows_missed) / n_cows # recall

0.8641975308641975

In [11]:
(n_cows - n_cows_false) / n_cows # precision

0.8395061728395061

In [43]:
map5095 = metrics.box.map.round(4)
map50 = metrics.box.map50.round(4)
precision = metrics.box.p[0].round(4)
recall = metrics.box.r[0].round(4)
f1 = metrics.box.f1[0].round(4)
# confusion matrix
conf_mat = metrics.confusion_matrix.matrix  # conf=0.25, iou_thres=0.45
n_all = conf_mat[:, 0].sum().astype(int)
n_missed = conf_mat[1, 0].sum().astype(int)
n_false = conf_mat[0, 1].sum().astype(int)
# write json
json = dict(
    map5095=map5095,
    map50=map50,
    precision=precision,
    recall=recall,
    f1=f1,
    n_all=n_all,
    n_missed=n_missed,
    n_false=n_false,
)
json

{'map5095': 0.574,
 'map50': 0.8945,
 'precision': 0.8529,
 'recall': 0.859,
 'f1': 0.8559,
 'n_all': 81,
 'n_missed': 11,
 'n_false': 13}

0.9761904761904762